Perform an intersubject correlation analysis on the categories. Each individuals betas are averaged over the 4 reps per category
to get a series of 180 beta values per vertex. The ISC is a leave-one-out correlation between a subject and the remaining 29 subject
group average. The final group ISC is the average of all the leave-one-out correlations. This is to replicate the results
of the HAD manuscript Figure 6 and validate our own preprocessing pipeline.

In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os
import sys
sys.path.append(os.getenv('PYTHONPATH')) 
import numpy as np
import pickle
import pandas as pd
import hcp_utils as hcp
import matplotlib.pyplot as plt
from nilearn import plotting

#local
from src.utils.helpers import vectorized_correlation

In [ ]:
dataset_root = os.path.join(os.getenv("DATASETS_ROOT", "/default/path/to/datasets"),"HumanActionsDataset") #use default if DATASETS_ROOT env variable is not set.
project_root = os.getenv("PROJECT_ROOT", "/default/path/to/project")
print(f"dataset_root: {dataset_root}")
print(f"project_root: {project_root}")
fmri_path = os.path.join(dataset_root,"derivatives", "GLM")
task='action'
dataset = 'HAD'
#first define the stimulus order and matrix.
#Next we will essentially place the betas into this pre-defined matrix
df = pd.read_table(os.path.join(dataset_root, "derivatives", "stimuli_metadata", "had_stiminfo.tsv"),index_col=False) #this stiminfo file is also saved in the MOSAIC repository
filenames = df['filename'].tolist()
cats = [f.split('_id_')[0].split('v_')[-1] for f in filenames]
categories = set(cats)
#get the order of categories
assert(len(categories) == 180)
subject_betas = {} #this will be a big dictionary holding all the beta estimates from the subjects
nvertices = 91282

In [ ]:

for sub in range(1,31):
    subject = f"sub-{sub:02}"
    print(f"loading betas for subject {subject}")
    #load the normalized betas for train and test
    with open(os.path.join(fmri_path, subject, "prepared_betas", f"{subject}_organized_betas_task-train_normalized.pkl"), 'rb') as f:
        betas_train, stimorder_train = pickle.load(f)
    with open(os.path.join(fmri_path, subject, "prepared_betas", f"{subject}_organized_betas_task-test_normalized.pkl"), 'rb') as f:
        betas_test, stimorder_test = pickle.load(f)

    #map the stimulus to categories in order
    categoryorder_train = []
    for stim in stimorder_train:
        tmp = stim.split('_id_')[0]
        cat = tmp.split('v_')[-1]
        categoryorder_train.append(cat)
    categoryorder_test = []
    for stim in stimorder_test:
        tmp = stim.split('_id_')[0]
        cat = tmp.split('v_')[-1]
        categoryorder_test.append(cat)

    #sort into categories
    beta_categories = np.zeros((len(categories), nvertices))
    for count, cat in enumerate(categories):
        cat_idx_train = np.isin(categoryorder_train, cat)
        cat_idx_test = np.isin(categoryorder_test, cat)
        assert(cat_idx_train.sum() == 3)
        assert(cat_idx_test.sum() == 1)

        betas_tmp = np.concatenate((betas_train[cat_idx_train,0,:], betas_test[cat_idx_test,0, :]), axis=0)
        assert(betas_tmp.shape[0] == 4)

        beta_categories[count, :] = np.mean(betas_tmp, axis=0)
    
    subject_betas.update({subject: beta_categories})

In [ ]:
nan_vertices_list = []
for subject, betas in subject_betas.items():
    sub_nans = np.argwhere(np.isnan(np.mean(betas, axis=0)))
    nan_vertices_list.extend(sub_nans.flatten())
    print(f"subject {subject} has {len(sub_nans)} nans")
nan_vertices = set(nan_vertices_list)
print(f"all subjects have {len(nan_vertices)} unique nans")


In [ ]:
isc = np.zeros((nvertices,))
subject_isc = {f"sub-{x:02}": 0 for x in range(1,31)}
for left_out_subject in subject_betas.keys():
    print(f"running isc on left out subject: {left_out_subject}")
    left_out_betas = subject_betas[left_out_subject]

    remaining_betas = np.zeros(left_out_betas.shape)
    n_remainingsubs = 0
    for key, value in subject_betas.items():
        if key != left_out_subject:
            n_remainingsubs += 1
            remaining_betas += value
    remaining_betas = remaining_betas/n_remainingsubs #average

    subject_corr = vectorized_correlation(left_out_betas, remaining_betas)
    subject_isc[left_out_subject] = subject_corr
    isc += subject_corr

isc = isc/len(subject_betas) #average the individual isc

In [ ]:
ext_list = ['png'] #only matters if save_flag is True 
save_flag=True #set to True to save plots or False to not save plots

save_root = os.path.join(project_root, "src", "fmriDatasetPreparation", "datasets", "HumanActionsDataset", "validation", "output", "GroupISC")
if not os.path.exists(save_root):
    os.makedirs(save_root)

stat = isc.copy()
print(f"Min, Max Group ISC pearson correlation: {np.nanmin(stat)}, {np.nanmax(stat)}")
threshold = None
cmap = 'coolwarm' #'hot'
#save inflated surfaces
cortex_data = hcp.cortex_data(stat)
#determine global min/max for consistent color scaling
datamin = np.nanmin(cortex_data)
datamax = np.nanmax(cortex_data)
vmin=-datamax #datamin
vmax=datamax

views = ['lateral', 'ventral', 'dorsal'] #['lateral', 'medial', 'dorsal', 'ventral', 'anterior', 'posterior']
for hemi in ['left','right']:
    mesh = hcp.mesh.inflated
    bg = hcp.mesh.sulc
    for view in views:
        display = plotting.plot_surf_stat_map(mesh, cortex_data, hemi=hemi,
        threshold=threshold, bg_map=bg, view=view, cmap=cmap)
        if save_flag:
            for ext in ext_list:
                if ext == 'png':
                    plt.savefig(os.path.join(save_root, f"groupISC_{dataset}_task-{task}_mesh-inflated_view-{view}_hemi-{hemi}.{ext}"),dpi=300)
                else:
                    plt.savefig(os.path.join(save_root, f"groupISC_{dataset}_task-{task}_mesh-inflated_view-{view}_hemi-{hemi}.{ext}"))

#Save flat maps. hemispheres are combined in one plot
#get the data for both hemispheres
cortex_data_left = hcp.left_cortex_data(stat)
cortex_data_right = hcp.right_cortex_data(stat)

#create a figure with multiple axes to plot each anatomical image
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 4), subplot_kw={'projection': '3d'})
plt.subplots_adjust(wspace=0)
im = plotting.plot_surf(hcp.mesh.flat_left, cortex_data_left,
        threshold=threshold, bg_map=hcp.mesh.sulc_left, 
        colorbar=False, cmap=cmap, 
        vmin=vmin, vmax=vmax,
        axes = axes[0])
im = plotting.plot_surf(hcp.mesh.flat_right, cortex_data_right,
        threshold=threshold, bg_map=hcp.mesh.sulc_right, 
        colorbar=False, cmap=cmap, 
        vmin=vmin, vmax=vmax,
        axes = axes[1])

#flip along the horizontal
axes[0].invert_yaxis()
axes[1].invert_yaxis()

#create colorbar
norm = plt.Normalize(vmin=vmin, vmax=vmax)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=axes.ravel().tolist(), shrink=0.6)

cbar.set_ticks([round(vmin,2), 0, round(vmax,2)])
cbar.set_ticklabels([round(vmin,2), 0, round(vmax,2)])
if save_flag:
    for ext in ext_list:
        if ext == 'png':
            plt.savefig(os.path.join(save_root, f"groupISC_{dataset}_task-{task}_mesh-flat.{ext}"),dpi=300)
        else:
            plt.savefig(os.path.join(save_root, f"groupISC_{dataset}_task-{task}_mesh-flat.{ext}"))
plt.show()

### Plot each left-out subject's correlation with the group average

In [ ]:
ext_list = ['png'] #only matters if save_flag is True 

save_flag=True #set to True to save plots or False to not save plots
threshold = None
cmap = 'coolwarm' #'hot'
save_root = os.path.join(project_root, "src", "fmriDatasetPreparation", "datasets", "HumanActionsDataset", "validation", "output", "SubjectISC")
if not os.path.exists(save_root):
    os.makedirs(save_root)
vmin= -0.75#-1 #fix vmin and vmax for all subjects because we want to compare
vmax=0.75 #1
views = [] #['lateral', 'medial'] #['lateral', 'medial', 'dorsal', 'ventral', 'anterior', 'posterior']
for left_out_subject, stat in subject_isc.items():
    print("*"*20)
    print(f"Min, Max subject {left_out_subject} ISC pearson correlation: {np.nanmin(stat)}, {np.nanmax(stat)}")
    #save inflated surfaces
    #determine global min/max for consistent color scaling
    cortex_data = hcp.cortex_data(stat)
    datamin = np.nanmin(cortex_data)
    datamax = np.nanmax(cortex_data)

    views = [] #['lateral', 'medial'] #['lateral', 'medial', 'dorsal', 'ventral', 'anterior', 'posterior']
    for hemi in ['left','right']:
        mesh = hcp.mesh.inflated
        bg = hcp.mesh.sulc
        for view in views:
            display = plotting.plot_surf_stat_map(mesh, cortex_data, hemi=hemi,
            threshold=threshold, bg_map=bg, view=view, cmap=cmap)
            if save_flag:
                for ext in ext_list:
                    if ext == 'png':
                        plt.savefig(os.path.join(save_root, f"{left_out_subject}_{dataset}_task-{task}_mesh-inflated_view-{view}_hemi-{hemi}_subjectISC.{ext}"),dpi=300)
                    else:
                        plt.savefig(os.path.join(save_root, f"{left_out_subject}_{dataset}_task-{task}_mesh-inflated_view-{view}_hemi-{hemi}_subjectISC.{ext}"))

    #Save flat maps. hemispheres are combined in one plot
    #get the data for both hemispheres
    cortex_data_left = hcp.left_cortex_data(stat)
    cortex_data_right = hcp.right_cortex_data(stat)

    #create a figure with multiple axes to plot each anatomical image
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 4), subplot_kw={'projection': '3d'})
    plt.subplots_adjust(wspace=0)
    im = plotting.plot_surf(hcp.mesh.flat_left, cortex_data_left,
            threshold=threshold, bg_map=hcp.mesh.sulc_left, 
            colorbar=False, cmap=cmap, 
            vmin=vmin, vmax=vmax,
            axes = axes[0])
    im = plotting.plot_surf(hcp.mesh.flat_right, cortex_data_right,
            threshold=threshold, bg_map=hcp.mesh.sulc_right, 
            colorbar=False, cmap=cmap, 
            vmin=vmin, vmax=vmax,
            axes = axes[1])

    #flip along the horizontal
    axes[0].invert_yaxis()
    axes[1].invert_yaxis()

    #create colorbar
    norm = plt.Normalize(vmin=vmin, vmax=vmax)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = fig.colorbar(sm, ax=axes.ravel().tolist(), shrink=0.6)

    cbar.set_ticks([round(vmin,2), round(datamin,2), 0, round(datamax,2), round(vmax,2)])
    cbar.set_ticklabels([round(vmin,2), round(datamin,2), 0, round(datamax,2), round(vmax,2)])
    if save_flag:
        for ext in ext_list:
            if ext == 'png':
                plt.savefig(os.path.join(save_root, f"{left_out_subject}_{dataset}_task-{task}_mesh-flat_subjectISC.{ext}"),dpi=300)
            else:
                plt.savefig(os.path.join(save_root, f"{left_out_subject}_{dataset}_task-{task}_mesh-flat_subjectISC.{ext}"))
    plt.show()